# IMPORTING LIBRARIES

In [36]:
import nltk
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import re
import pandas as pd
import numpy as np

In [37]:
df = pd.read_csv(r'C:\Users\prach\Desktop\ML projects\NLP - Krish Naik\Data.csv', encoding = "ISO-8859-1")

# DATA EXPLORATION

In [38]:
df.head(2)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2000-01-03,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,...,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
1,2000-01-04,0,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,...,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4101 entries, 0 to 4100
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    4101 non-null   object
 1   Label   4101 non-null   int64 
 2   Top1    4101 non-null   object
 3   Top2    4101 non-null   object
 4   Top3    4101 non-null   object
 5   Top4    4101 non-null   object
 6   Top5    4101 non-null   object
 7   Top6    4101 non-null   object
 8   Top7    4101 non-null   object
 9   Top8    4101 non-null   object
 10  Top9    4101 non-null   object
 11  Top10   4101 non-null   object
 12  Top11   4101 non-null   object
 13  Top12   4101 non-null   object
 14  Top13   4101 non-null   object
 15  Top14   4101 non-null   object
 16  Top15   4101 non-null   object
 17  Top16   4101 non-null   object
 18  Top17   4101 non-null   object
 19  Top18   4101 non-null   object
 20  Top19   4101 non-null   object
 21  Top20   4101 non-null   object
 22  Top21   4101 non-null   

Class 1- STock price is increased 
Class 0 - Stock price stays same or decreased

# SPLITTING INTO TRAIN AND TEST SETS

In [40]:
#converting the entire datatset into train and test 
train = df[df['Date'] < '20150101']
test = df[df['Date'] > '20141231']

In [41]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3975 entries, 0 to 3974
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    3975 non-null   object
 1   Label   3975 non-null   int64 
 2   Top1    3975 non-null   object
 3   Top2    3975 non-null   object
 4   Top3    3975 non-null   object
 5   Top4    3975 non-null   object
 6   Top5    3975 non-null   object
 7   Top6    3975 non-null   object
 8   Top7    3975 non-null   object
 9   Top8    3975 non-null   object
 10  Top9    3975 non-null   object
 11  Top10   3975 non-null   object
 12  Top11   3975 non-null   object
 13  Top12   3975 non-null   object
 14  Top13   3975 non-null   object
 15  Top14   3975 non-null   object
 16  Top15   3975 non-null   object
 17  Top16   3975 non-null   object
 18  Top17   3975 non-null   object
 19  Top18   3975 non-null   object
 20  Top19   3975 non-null   object
 21  Top20   3975 non-null   object
 22  Top21   3975 non-null   

In [42]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 378 entries, 3723 to 4100
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    378 non-null    object
 1   Label   378 non-null    int64 
 2   Top1    378 non-null    object
 3   Top2    378 non-null    object
 4   Top3    378 non-null    object
 5   Top4    378 non-null    object
 6   Top5    378 non-null    object
 7   Top6    378 non-null    object
 8   Top7    378 non-null    object
 9   Top8    378 non-null    object
 10  Top9    378 non-null    object
 11  Top10   378 non-null    object
 12  Top11   378 non-null    object
 13  Top12   378 non-null    object
 14  Top13   378 non-null    object
 15  Top14   378 non-null    object
 16  Top15   378 non-null    object
 17  Top16   378 non-null    object
 18  Top17   378 non-null    object
 19  Top18   378 non-null    object
 20  Top19   378 non-null    object
 21  Top20   378 non-null    object
 22  Top21   378 non-null  

# TEXT PREPROCESSING

In [43]:
#text preprocessing 

#for all columns from 2 to 27, any charcater other than aphabets wi be replaced with empty string 
data=train.iloc[:,2:27]
data.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

In [45]:
# Renaming column names 
list1= [i for i in range(25)]
new_Index=[str(i) for i in list1]
data.columns= new_Index
data.head(2)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,A hindrance to operations extracts from the...,Scorecard,Hughes instant hit buoys Blues,Jack gets his skates on at ice cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,Derby raise a glass to Strupar s debut double,Southgate strikes Leeds pay the penalty,...,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl s successor drawn into scandal,The difference between men and women,Sara Denver nurse turned solicitor,Diana s landmine crusade put Tories in a panic,Yeltsin s resignation caught opposition flat f...,Russian roulette,Sold out,Recovering a title
1,Scorecard,The best lake scene,Leader German sleaze inquiry,Cheerio boyo,The main recommendations,Has Cubie killed fees,Has Cubie killed fees,Has Cubie killed fees,Hopkins furious at Foster s lack of Hannibal...,Has Cubie killed fees,...,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man s extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn t know without the ...,Millennium bug fails to bite


In [46]:
# Convertng headlines to lower case
for i in new_Index:
    data[i]=data[i].str.lower()
data.head(2)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,a hindrance to operations extracts from the...,scorecard,hughes instant hit buoys blues,jack gets his skates on at ice cold alex,chaos as maracana builds up for united,depleted leicester prevail as elliott spoils e...,hungry spurs sense rich pickings,gunners so wide of an easy target,derby raise a glass to strupar s debut double,southgate strikes leeds pay the penalty,...,flintoff injury piles on woe for england,hunters threaten jospin with new battle of the...,kohl s successor drawn into scandal,the difference between men and women,sara denver nurse turned solicitor,diana s landmine crusade put tories in a panic,yeltsin s resignation caught opposition flat f...,russian roulette,sold out,recovering a title
1,scorecard,the best lake scene,leader german sleaze inquiry,cheerio boyo,the main recommendations,has cubie killed fees,has cubie killed fees,has cubie killed fees,hopkins furious at foster s lack of hannibal...,has cubie killed fees,...,on the critical list,the timing of their lives,dear doctor,irish court halts ira man s extradition to nor...,burundi peace initiative fades after rebels re...,pe points the way forward to the ecb,campaigners keep up pressure on nazi war crime...,jane ratcliffe,yet more things you wouldn t know without the ...,millennium bug fails to bite


In [47]:
#here every row will be converted into one text
headlines = []
for row in range(0,len(data.index)):
    headlines.append(' '.join(str(x) for x in data.iloc[row,0:25]))

In [48]:
#printing the first row 
headlines[0]

'a  hindrance to operations   extracts from the leaked reports scorecard hughes  instant hit buoys blues jack gets his skates on at ice cold alex chaos as maracana builds up for united depleted leicester prevail as elliott spoils everton s party hungry spurs sense rich pickings gunners so wide of an easy target derby raise a glass to strupar s debut double southgate strikes  leeds pay the penalty hammers hand robson a youthful lesson saints party like it s      wear wolves have turned into lambs stump mike catches testy gough s taunt langer escapes to hit     flintoff injury piles on woe for england hunters threaten jospin with new battle of the somme kohl s successor drawn into scandal the difference between men and women sara denver  nurse turned solicitor diana s landmine crusade put tories in a panic yeltsin s resignation caught opposition flat footed russian roulette sold out recovering a title'

# COUNTVECTORIZER AND RANDOM FOREST CLASSIFIER


In [82]:
from sklearn.feature_extraction.text import CountVectorizer  #takes sentences into vectors of features
from sklearn.ensemble import RandomForestClassifier

#Bag of words
countvector=CountVectorizer(ngram_range=(2,2))
traindataset=countvector.fit_transform(headlines)

#RandomForest Classifier for train dataset
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier.fit(traindataset,train['Label'])  #traindataset is y_pred and train[label] is dependent feature

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [83]:
#Predict for the Test Dataset
test_transform= []

for row in range(0,len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row,2:27]))
    
test_dataset = countvector.transform(test_transform)
predictions = randomclassifier.predict(test_dataset)

In [103]:
#Calculate accuracy
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score


matrix=confusion_matrix(test['Label'],predictions)
print('The confusion Matrix is',matrix)

score1=accuracy_score(test['Label'],predictions)
print('The Accuracy of the Random forest classifier model is:',score1*100)


The confusion Matrix is [[141  45]
 [  7 185]]
The Accuracy of the Random forest classifier model is: 86.24338624338624


# COUNTVECTORIZER AND NAIVE BAYES


In [85]:
from sklearn.naive_bayes import MultinomialNB
detect_model = MultinomialNB().fit(traindataset, train['Label'])

#Predict for the Test Dataset
test_transform_NB= []
for row in range(0,len(test.index)):
    test_transform_NB.append(' '.join(str(x) for x in test.iloc[row,2:27]))
    
    
test_dataset_NB = countvector.transform(test_transform_NB)
#predictions_NB = randomclassifier.predict(test_dataset_NB)
y_pred=detect_model.predict(test_dataset_NB)

In [102]:
#Calculate accuracy
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score


matrix=confusion_matrix(test['Label'],y_pred)
print('The confusion Matrix is',matrix)

score2=accuracy_score(test['Label'],y_pred)
print('The Accuracy of the Random forest classifier model is:',score2*100)

The confusion Matrix is [[138  48]
 [ 10 182]]
The Accuracy of the Random forest classifier model is: 84.65608465608466


# COUNTVECTORIZER AND PASSIVE AGGRESSIVE CLASSIFIER

In [ ]:
We can simply say that an online-learning algorithm will get
    a training example, update the classifier, and then throw away the example.
    
How Passive-Aggressive Algorithms Work:
Passive-Aggressive algorithms are called so because :
    Passive: If the prediction is correct, keep the model and do not make any changes. i.e., the data in the example is not enough to cause any changes in the model. 
    Aggressive: If the prediction is incorrect, make changes to the model. i.e., some change to the model may correct it.
        


In [92]:
from sklearn.linear_model import PassiveAggressiveClassifier
#from sklearn.datasets import make_classification
AC_model =  PassiveAggressiveClassifier(C = 0.5, random_state = 5) 
AC_model.fit(traindataset, train['Label'])

#y_pred = AC_model.predict(test)

PassiveAggressiveClassifier(C=0.5, random_state=5)

In [93]:
#Predict for the Test Dataset
test_transform_AC= []
for row in range(0,len(test.index)):
    test_transform_AC.append(' '.join(str(x) for x in test.iloc[row,2:27]))

In [99]:
test_dataset_AC = countvector.transform(test_transform_AC)
y_pred_AC=AC_model.predict(test_dataset_AC)

In [104]:
#Calculate accuracy
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score


matrix=confusion_matrix(test['Label'],y_pred_AC)
print('The confusion Matrix is',matrix)

score3=accuracy_score(test['Label'],y_pred_AC)
print('The Accuracy of the Random forest classifier model is:',score3*100)

The confusion Matrix is [[158  28]
 [ 33 159]]
The Accuracy of the Random forest classifier model is: 83.86243386243386


In [106]:
print("Accuracy of Random Forest Model: ",score1*100)
print("Accuracy of Naive Bayes Model: ",score2*100)
print("Accuracy of Passive Aggressive Model:",score3*100)

Accuracy of Random Forest Model:  86.24338624338624
Accuracy of Naive Bayes Model:  84.65608465608466
Accuracy of Passive Aggressive Model: 83.86243386243386
